<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

#  Mind Utils Generation

Many news recommendation methods ultilize word embeddings, news vertical embeddings, news subvertical embeddings and user id embedding. Therefore, it is necessary to generate a word dictionary, a vertical dictionary, a subvertical dictionary and a userid dictionary to convert words, news verticals, subvericals and user ids from strings to indexes. To use the pretrain word embedding, a embedding matrix is generated as the intial weight of the word embedding layer.


This notebook gives examples about how to generate
* word_dict.pkl: convert the words in news titles into indexes.
* word_dict_all.pkl: convert the words in news titles and abstracts into indexes.
* embedding.npy: pretrained word embedding matrix of words in word_dict.pkl
* embedding_all.npy: pretrained embedding matrix of words in word_dict_all.pkl
* vert_dict.pkl: convert news verticals into indexes.
* subvert_dict.pkl: convert news subverticals into indexes.
* uid2index.pkl: convert user ids into indexes.

In [22]:
import sys
sys.path.append("../../")
import os
import papermill as pm
print("System version: {}".format(sys.version))

from tempfile import TemporaryDirectory
from reco_utils.dataset.mind import (download_mind,
                                     extract_mind,
                                     _download_and_extract_globe)
from reco_utils.dataset.download_utils import unzip_file

System version: 3.6.11 | packaged by conda-forge | (default, Aug  5 2020, 20:09:42) 
[GCC 7.5.0]


In [2]:
mind_type="small"
# word_embedding_dim should be in [50, 100, 200, 300]
word_embedding_dim = 300

In [3]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name
train_zip, valid_zip = download_mind(size=mind_type, dest_path=data_path)
unzip_file(train_zip, os.path.join(data_path, 'train'))
unzip_file(valid_zip, os.path.join(data_path, 'valid'))
output_path = os.path.join(data_path, 'utils')
os.makedirs(output_path, exist_ok=True)

100%|██████████| 51.7k/51.7k [00:01<00:00, 40.2kKB/s]
100%|██████████| 30.2k/30.2k [00:00<00:00, 35.8kKB/s]


## Prepare utils of news

* word dictionary
* vertical dictionary
* subvetical dictionary

In [4]:
import pandas as pd
import re
from collections import Counter
from tqdm import tqdm
import pickle

In [5]:
news = pd.read_table(os.path.join(data_path, 'train', 'news.tsv'),
                     names=['newid', 'vertical', 'subvertical', 'title',
                            'abstract', 'url', 'entities in title', 'entities in abstract'],
                     usecols = ['vertical', 'subvertical', 'title', 'abstract'])

In [6]:
news.head()

,vertical,subvertical,title,abstract
0,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."


In [7]:
news_vertical = news.vertical.drop_duplicates().reset_index(drop=True)
vert_dict_inv = news_vertical.to_dict()
vert_dict = {v: k+1 for k, v in vert_dict_inv.items()}

news_subvertical = news.subvertical.drop_duplicates().reset_index(drop=True)
subvert_dict_inv = news_subvertical.to_dict()
subvert_dict = {v: k+1 for k, v in vert_dict_inv.items()}

In [8]:
def word_tokenize(sent):
    pat = re.compile(r'[\w]+|[.,!?;|]')
    if isinstance(sent, str):
        return pat.findall(sent.lower())
    else:
        return []

In [9]:
news.title = news.title.apply(word_tokenize)
news.abstract = news.abstract.apply(word_tokenize)

In [10]:
word_cnt = Counter()
word_cnt_all = Counter()

for i in tqdm(range(len(news))):
    word_cnt.update(news.loc[i]['title'])
    word_cnt_all.update(news.loc[i]['title'])
    word_cnt_all.update(news.loc[i]['abstract'])

100%|██████████| 51282/51282 [00:27<00:00, 1854.28it/s]


In [11]:
word_dict = {k: v+1 for k, v in zip(word_cnt, range(len(word_cnt)))}
word_dict_all = {k: v+1 for k, v in zip(word_cnt_all, range(len(word_cnt_all)))}

In [12]:
with open(os.path.join(output_path, 'vert_dict.pkl'), 'wb') as f:
    pickle.dump(vert_dict, f)
    
with open(os.path.join(output_path, 'subvert_dict.pkl'), 'wb') as f:
    pickle.dump(subvert_dict, f)

with open(os.path.join(output_path, 'word_dict.pkl'), 'wb') as f:
    pickle.dump(word_dict, f)
    
with open(os.path.join(output_path, 'word_dict_all.pkl'), 'wb') as f:
    pickle.dump(word_dict, f)

## Prepare embedding matrixs
* embedding.npy
* embedding_all.npy

In [13]:
import numpy as np

In [14]:
glove_path = _download_and_extract_globe(data_path)

100%|██████████| 842k/842k [09:21<00:00, 1.50kKB/s]  


In [15]:
def load_matrix(path_emb, word_dict, word_embedding_dim):
    embedding_matrix = np.zeros((len(word_dict)+1, word_embedding_dim))
    exist_word=[]

    with open(os.path.join(path_emb, f"glove.6B.{word_embedding_dim}d.txt"),'rb') as f:
        for l in tqdm(f):
            l=l.split()
            word = l[0].decode()
            if len(word) != 0:
                wordvec = [float(x) for x in l[1:]]
                if word in word_dict:
                    index = word_dict[word]
                    embedding_matrix[index]=np.array(wordvec)
                    exist_word.append(word)
                    
    return embedding_matrix, exist_word

In [16]:
embedding_matrix, exist_word = load_matrix(glove_path, word_dict, word_embedding_dim)
embedding_all_matrix, exist_all_word = load_matrix(glove_path, word_dict_all, word_embedding_dim)

400000it [00:29, 13535.85it/s]
400000it [00:29, 13444.48it/s]


In [17]:
np.save(os.path.join(output_path, 'embedding.npy'), embedding_matrix)
np.save(os.path.join(output_path, 'embedding_all.npy'), embedding_all_matrix)

## Prepare uid2index.pkl

In [18]:
uid2index = {}

with open(os.path.join(data_path, 'train', 'behaviors.tsv'), 'r') as f:
    for l in tqdm(f):
        uid = l.strip('\n').split('\t')[1]
        if uid not in uid2index:
            uid2index[uid] = len(uid2index) + 1

156965it [00:00, 455714.84it/s]


In [19]:
with open(os.path.join(output_path, 'uid2index.pkl'), 'wb') as f:
    pickle.dump(uid2index, f)

In [20]:
utils_state = {
    'vert_num': len(vert_dict),
    'subvert_num': len(subvert_dict),
    'word_num': len(word_dict),
    'word_num_all': len(word_dict_all),
    'embedding_exist_num': len(exist_word),
    'embedding_exist_num_all': len(exist_all_word),
    'uid2index': len(uid2index)
}

In [23]:
pm.record("utils_state", utils_state)

/anaconda/envs/reco_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  """Entry point for launching an IPython kernel.
